In [ ]:
import numpy as np
import random

# read the gif masks
import imageio

import matplotlib.pyplot as plt
import pydicom as dicom
import matplotlib.pylab as plb

from PIL import Image

import cv2

import glob

# install the library using this command: pip install git+https://github.com/tensorflow/docs
import tensorflow_docs.vis.embed as embed

In [ ]:
pixel_data = []
paths = glob.glob("C:/Users/arthu/Downloads/data/NN_Pieds/Controles/pied_50/pied_50/pied_entier_50/*.dcm")

for path in paths:
    dataset = dicom.dcmread(path)
    pixel_data.append(dataset.pixel_array)

In [ ]:
images = np.array(pixel_data) # (741, 512, 512)
img = np.transpose(images, (1, 2, 0)) # (512, 512, 741)

In [ ]:
pixel_data = []
paths = glob.glob("C:/Users/arthu/Downloads/data/NN_Pieds/Controles/pied_50/pied_50/calcaneum/*.bmp")

for path in paths:
    mask = Image.open(path)
    pixel_data.append(np.array(mask))

In [ ]:
masks = np.array(pixel_data) # (740, 512, 512)
msks = np.transpose(masks, (1, 2, 0)) # (512, 512, 740)

In [ ]:
temp = 0
non_zero = []
for mask in masks:
    somme = mask.sum()
    if somme > 0:
        non_zero.append(mask)
        
    temp += somme

print(temp)

In [ ]:
non_zero_masks = np.array(non_zero)

In [ ]:
non_zero_masks.shape

In [ ]:
def sample_stack(stack, rows=6, cols=6, start_with=10, show_every=3):
    fig,ax = plt.subplots(rows,cols,figsize=[20,22])
    for i in range(rows*cols):
        ind = start_with + i*show_every
        ax[int(i/rows),int(i % rows)].set_title(f'slice {ind}')
        ax[int(i/rows),int(i % rows)].imshow(stack[ind],cmap='gray')
        ax[int(i/rows),int(i % rows)].axis('off')
    plt.show()

In [ ]:
sample_stack(images, rows=5, cols=5, start_with=0, show_every = 29)

In [ ]:
sample_stack(non_zero_masks, rows=5, cols=5, start_with=0, show_every = 7)

In [ ]:
data_number = '01'

# create the gif

# images
#imageio.mimsave('./images_notebooks/gif/Foot_test_CTscan_{}.gif'.format(data_number), images, duration=0.1)

# masks
imageio.mimsave('./images_notebooks/gif/Foot_test_mask_annotated_{}.gif'.format(data_number), non_zero_masks, duration=0.1)

In [ ]:
# plot the gif for the CT scans
embed.embed_file('./images_notebooks/gif/Foot_test_CTscan_{}.gif'.format(data_number))

In [ ]:
# plot the gif for the annotated masks
embed.embed_file('./images_notebooks/gif/Foot_test_mask_annotated_{}.gif'.format(data_number))

convert image format

In [ ]:

image_path = 'blablabla'
image_format = '.jpg' # or '.png'
image_path = image_path.replace('.dcm', image_format)

cv2.imwrite(image_path, pixel_array_numpy)

# MANUAL SEGMENTING

In [ ]:
from skimage.segmentation import clear_border
from skimage import measure
from skimage.measure import label, regionprops

from scipy import ndimage as ndi
from scipy.ndimage import measurements, center_of_mass, binary_dilation, zoom

import plotly.graph_objects as go

import os
import imageio

import nibabel as nib

In [ ]:
plt.imshow(images[300], cmap='gray')
plt.colorbar()
plt.show()

In [ ]:
mask = images < 250

In [ ]:
mask = mask.astype(np.uint8)

In [ ]:
plt.imshow(mask[300])
plt.colorbar()
plt.show()

In [ ]:
b = cv2.getStructuringElement(cv2.MORPH_CROSS, ksize=(5,5))

b

In [ ]:
erosion_test = cv2.erode(mask[300], kernel=b, borderType=cv2.BORDER_REFLECT)

In [ ]:
dilation_test = cv2.dilate(mask[300], kernel=b, borderType=cv2.BORDER_REFLECT)

In [ ]:
plt.imshow(erosion_test)
plt.colorbar()
plt.show()

In [ ]:
mask = np.vectorize(clear_border, signature='(n,m)->(n,m)')(mask)
plt.imshow(mask[300])
plt.colorbar()
plt.show()

In [ ]:
mask_labeled = np.vectorize(label, signature='(n,m)->(n,m)')(mask)
plt.imshow(mask_labeled[300])
plt.colorbar()
plt.show()

In [ ]:
mask = mask_labeled > 0

In [ ]:
mask = np.vectorize(ndi.binary_fill_holes, signature='(n,m)->(n,m)')(mask)
plt.imshow(mask[300])
plt.colorbar()
plt.show()

In [ ]:
# create the gif
interm = 1*mask # convert array of bool to array of int32
imageio.mimsave('./images_notebooks/gif/Foot_test_mask_{}.gif'.format(data_number), interm, duration=0.1)

In [ ]:
# plot the gif
embed.embed_file('./images_notebooks/gif/Foot_test_mask_{}.gif'.format(data_number))